In [138]:
## Imports
import trimesh
import open3d as o3d
import numpy as np
from sklearn.cluster import KMeans
import os
import argparse
import json

from utils import extract_largest_cavity, extract_cavity_parts ,extract_top_percentage
from utils import show_mesh_dimensions_with_cylinders, visualize_roughness, create_cylinder_between_points
from utils import split_side_and_get_normal_means, calculate_roughness, discrete_mean_curvature_measure_gpu


BOTTOM_THRESHOLD_PERCENTAGE= 0.4
MEAN_CURVATURE_RADİUS= 2
TOOTH_PERCENTAGE = 28.0
CAVITY_PERCENTAGE = 20.0



In [148]:

### Load and compute the mean curvature
# Load the tooth STL model using Trimesh
mesh_trimesh = trimesh.load_mesh(f"StudentTeeth/Mehmet.stl")

# Get vertices, faces, and normals
old_vertices = np.array(mesh_trimesh.vertices)
old_faces = np.array(mesh_trimesh.faces)
old_normals = np.array(mesh_trimesh.vertex_normals)

old_tooth_o3d = o3d.geometry.TriangleMesh()
old_tooth_o3d.vertices = o3d.utility.Vector3dVector(old_vertices)
old_tooth_o3d.triangles = o3d.utility.Vector3iVector(old_faces)
old_tooth_o3d.compute_vertex_normals()# Convert full tooth to Open3D mesh

old_tooth_o3d.paint_uniform_color([0.8, 0.8, 0.8])  # light gray
# Visualize with both bounding boxes and the original mesh
# Compute Mean Curvature using Trimesh

TriangleMesh with 4651 points and 9174 triangles.

In [149]:
num_vertices = old_vertices.shape[0]
num_samples = int(num_vertices * 0.2)

# Sort vertices by Z-coordinate (descending order)
sorted_indices = np.argsort(old_vertices[:, 2])[::-1]  # Use `1` for Y-axis or `0` for X-axis
top_indices = sorted_indices[:num_samples]  # Select top 10%

# Extract the top points
top_points = old_vertices[top_indices]

# Create a point cloud object
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(top_points)

# Oriented Bounding Box (OBB) - if you want tighter fit but possibly rotated
obb = pcd.get_oriented_bounding_box()
obb.color = (0, 1, 0)  # Green

R = obb.R              # Rotation matrix (3x3)
center = obb.center    # Center of OBB


T_translate_to_origin = np.eye(4)
T_translate_to_origin[:3, 3] = -center

T_translate_back = np.eye(4)
T_translate_back[:3, 3] = center
# Create rotation matrix in 4x4
T_rotate = np.eye(4)
T_rotate[:3, :3] = R  # your rotation matrix
T_final = T_translate_back @ T_rotate @ T_translate_to_origin

mesh_trimesh.apply_transform(T_final)
#rotate tooth_o3d 

world_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=5.0, origin=[0, 0, 5])

vertices = np.array(mesh_trimesh.vertices)
faces = np.array(mesh_trimesh.faces)
normals = np.array(mesh_trimesh.vertex_normals)

tooth_o3d = o3d.geometry.TriangleMesh()
tooth_o3d.vertices = o3d.utility.Vector3dVector(vertices)
tooth_o3d.triangles = o3d.utility.Vector3iVector(faces)
tooth_o3d.compute_vertex_normals()# Convert full tooth to Open3D mesh

tooth_o3d.paint_uniform_color([1, 0.8, 0.8])  # light gray
# Visualize with both bounding boxes and the original mesh

TriangleMesh with 4651 points and 9174 triangles.

In [150]:
mean_curvature = trimesh.curvature.discrete_mean_curvature_measure(mesh_trimesh, mesh_trimesh.vertices, MEAN_CURVATURE_RADİUS)
### kavitenin seçilmesi 
cavity_indices = np.where(mean_curvature < 0.4)[0]  # Select all vertices with negative curvature
largest_cavity_mesh, largest_cavity_indices = extract_largest_cavity(vertices, faces, cavity_indices)
cavity_vertices= np.asarray(largest_cavity_mesh.vertices)

top_tooth_mesh = extract_top_percentage(tooth_o3d , TOOTH_PERCENTAGE) 
top_cavity_mesh = extract_top_percentage(largest_cavity_mesh , CAVITY_PERCENTAGE)
tooth_dimension_cylinder_meshes, tooth_width, tooth_length = show_mesh_dimensions_with_cylinders(top_tooth_mesh)
cavity_dimension_cylinder_meshes, cavity_width, cavity_length = show_mesh_dimensions_with_cylinders(largest_cavity_mesh)


### Kavitenin alt kısmının seçilmesi ve kavite yüksekliğinin hesaplanması
side_bottom, cavity_bottom = extract_cavity_parts(largest_cavity_mesh, BOTTOM_THRESHOLD_PERCENTAGE)

# kavite altının z eksenindeki ortalamasını al
bottom_vertices = np.asarray(cavity_bottom.vertices)
bottom_z_values = bottom_vertices[:, 2]
min_z_mean = np.mean(bottom_z_values) 

# kavite alanının en üstü 
max_z = np.max(cavity_vertices[:, 2])
cavity_depth = max_z - min_z_mean  # Derinlik (Z eksenindeki fark)

outline_indices = np.where((mean_curvature > 3.0))[0]
roughness = calculate_roughness(cavity_bottom)
colored_roughness = visualize_roughness(cavity_bottom,tooth_o3d)

cavity_centroid = np.mean(cavity_vertices, axis=0)
min_z_point = [cavity_centroid[0], cavity_centroid[1], min_z_mean]
max_z_point = [cavity_centroid[0], cavity_centroid[1], max_z]
cavity_depth_mesh = create_cylinder_between_points(min_z_point, max_z_point)

right_mesh, left_mesh, right_normal_mean, left_normal_mean = split_side_and_get_normal_means(side_bottom)

cavity_bottom.compute_vertex_normals()
bottom_normal_mean = np.mean(np.asarray(cavity_bottom.vertex_normals),axis=0)

right_angle = np.dot(right_normal_mean, bottom_normal_mean)
left_angle = np.dot(left_normal_mean, bottom_normal_mean)


In [156]:
o3d.visualization.draw_geometries([largest_cavity_mesh,tooth_o3d , old_tooth_o3d],mesh_show_back_face=True)

In [242]:

def get_highest_point_near_mid_y(mesh, axis,tol=1 , mesial = 1): # mesial = 1 mesial = -1
    new_vertices = np.array(mesh.vertices)
    num_vertices = new_vertices.shape[0]
    num_samples = int(num_vertices * 0.05)

    # Sort vertices by Z-coordinate (descending order)
    sorted_indices = np.argsort(new_vertices[:, axis ])[::mesial]  # Use `1` for Y-axis or `0` for X-axis
    indices = sorted_indices[:num_samples]  # Select

    # Extract the top points
    points = new_vertices[indices]
    min_y = np.min(points[:,1])
    max_y = np.max(points[:,1]) /1.5
    mid_y = (min_y + max_y) / 2.0

    close_to_mid_y = points[(points[:,1] >= mid_y - tol) & (points[:,1] <= mid_y + tol)]
    
    highest_point_idx = np.argmin(close_to_mid_y[:,2])
    highest_point = close_to_mid_y[highest_point_idx]

    # Open3D PointCloud olarak döndür
    highest_pcd = o3d.geometry.PointCloud()
    highest_pcd.points = o3d.utility.Vector3dVector([highest_point])
    return highest_pcd

In [246]:
def calculate_oklidian_length_point(point1 , point2 ): 
    x1 = np.asarray(point1.points)[0]
    x2 = np.asarray(point2.points)[0]

    # Öklidyen mesafe hesapla
    distance = np.linalg.norm(x1 - x2)
    return distance

outer_mesial_point = get_highest_point_near_mid_y(tooth_o3d , 0 , mesial=1) 
cavity_mesial_point = get_highest_point_near_mid_y(largest_cavity_mesh , 0 , mesial=1) 
outer_distal_point = get_highest_point_near_mid_y(tooth_o3d , 0 , mesial=-1) 
cavity_distal_point = get_highest_point_near_mid_y(largest_cavity_mesh , 0 , mesial=-1) 

mesial_marginal_ridge_width = calculate_oklidian_length_point(outer_mesial_point, cavity_mesial_point )
distal_marginal_ridge_width = calculate_oklidian_length_point(outer_distal_point, cavity_distal_point )

print(mesial_marginal_ridge_width)

o3d.visualization.draw_geometries([cavity_mesial_point ,tooth_o3d,world_frame],mesh_show_back_face=False)


1.483559854489878
